In [1]:
import pandas as pd
import xml.etree.cElementTree as et
import csv
import os
from collections import defaultdict
from lxml import etree

In [3]:
%matplotlib inline

import numpy as np 
import sklearn
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split


/Users/sarthakjagetia/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/sarthakjagetia/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
###THIS BLOCK OF CODE READS THE CSV FILES THAT ARE GENERATED BY OUR XML PARSER
#YOU MIGHT HAVE TO CHANGE THE DIRECTORY NAME ACCORDING TO THE FILE YOU PASTED IN...
op_data1 = pd.read_csv('operational_data/test_xmldata.2017-07-04.csv')
op_data1 = op_data1.dropna(axis=0, how='all')
op_data1.head()

/Users/sarthakjagetia/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,devicename,entry_timestamp,exit_timestamp,ole,owi,ohe,speed,incr,otl,oga
0,PS06Test,2017-07-03T23:57:58.264,2017-07-03T23:58:00.334,15.0,8.0,6.8,529.92,24665.0,16.14,42.8
1,PS06Test,2017-07-03T23:58:00.129,2017-07-03T23:58:02.224,16.8,24.2,6.6,529.92,27235.0,17.72,46.8
2,PS06Test,2017-07-03T23:58:02.338,2017-07-03T23:58:04.407,13.8,12.4,8.2,529.92,30210.0,15.35,55.6
3,PS06Test,2017-07-03T23:58:04.206,2017-07-03T23:58:06.358,23.8,12.8,6.4,529.92,32870.0,24.41,47.0
4,PS06Test,2017-07-03T23:58:06.479,2017-07-03T23:58:08.633,24.2,12.6,6.6,529.92,35970.0,24.41,55.8


In [5]:
#THIS BLOCK OF CODE CALCULATES THE VOLUME FROM THE DIMENSIONS GIVEN IN THE DATA 
op_data1['volume'] = 0
for index, row in op_data1.iterrows():
    #print((row['volume']))
    res = row['ole']*row['owi']*row['ohe'] #OLE = OBJECT LENGTH; OHE = OBJECT HEIGHT; OWI = OBJECT WIDTH
    op_data1.loc[index,'volume'] = res
op_data1.head()

,devicename,entry_timestamp,exit_timestamp,ole,owi,ohe,speed,incr,otl,oga,volume
0,PS06Test,2017-07-03T23:57:58.264,2017-07-03T23:58:00.334,15.0,8.0,6.8,529.92,24665.0,16.14,42.8,816.000
1,PS06Test,2017-07-03T23:58:00.129,2017-07-03T23:58:02.224,16.8,24.2,6.6,529.92,27235.0,17.72,46.8,2683.296
2,PS06Test,2017-07-03T23:58:02.338,2017-07-03T23:58:04.407,13.8,12.4,8.2,529.92,30210.0,15.35,55.6,1403.184
3,PS06Test,2017-07-03T23:58:04.206,2017-07-03T23:58:06.358,23.8,12.8,6.4,529.92,32870.0,24.41,47.0,1949.696
4,PS06Test,2017-07-03T23:58:06.479,2017-07-03T23:58:08.633,24.2,12.6,6.6,529.92,35970.0,24.41,55.8,2012.472


In [6]:
#DROPPING ALL THE COLUMNS WHICH ARE NOT USEFUL FOR KNN ANALYSIS 
op_data = op_data1.drop(['devicename', 'entry_timestamp','exit_timestamp'], axis=1)
op_data.head()

,ole,owi,ohe,speed,incr,otl,oga,volume
0,15.0,8.0,6.8,529.92,24665.0,16.14,42.8,816.000
1,16.8,24.2,6.6,529.92,27235.0,17.72,46.8,2683.296
2,13.8,12.4,8.2,529.92,30210.0,15.35,55.6,1403.184
3,23.8,12.8,6.4,529.92,32870.0,24.41,47.0,1949.696
4,24.2,12.6,6.6,529.92,35970.0,24.41,55.8,2012.472


In [7]:
#THIS FINDS THE CORRELATION BETWEEN THE COLUMNS OF THE DATA
#df['A'].corr(df['B'])
op_data['volume'].corr(op_data['otl'])

0.65658631955198243

In [8]:
#CONVERTING ALL THE VALUES IN DATAFRAME TO INTEGER
op_data = op_data.astype('int')
op_data.head()

,ole,owi,ohe,speed,incr,otl,oga,volume
0,15,8,6,529,24665,16,42,816
1,16,24,6,529,27235,17,46,2683
2,13,12,8,529,30210,15,55,1403
3,23,12,6,529,32870,24,47,1949
4,24,12,6,529,35970,24,55,2012


In [10]:
# Splitting data into train and test datasets, with 7:3 ratio
#I AM RUNNING KNN FOR Y = VOLUME AND X = DATASET
y = op_data['volume'] #oga default
x = op_data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [11]:
#DETERMINING K NEAREST NEIGHBOURS!
#RUNNING KNeighborsClassifier FOR K VALUES 1 TO 30 TO SEE WHICH K VALUE RESULTS MAX ACCURACY
for k in range (1,30):
    knn = KNeighborsClassifier(n_neighbors=k) 
    knn.fit(x_train,y_train)
    y_pred = knn.predict(x_test)
    print('k = {}'.format(k))
    print('Accuracy on test data: {}'.format(knn.score(x_test, y_test)))
    print('======================================================')

k = 1
Accuracy on test data: 0.008544795442775763
k = 2
Accuracy on test data: 0.0072501294665976174
k = 3
Accuracy on test data: 0.004919730709476955
k = 4
Accuracy on test data: 0.002589331952356292
k = 5
Accuracy on test data: 0.0020714655618850335
k = 6
Accuracy on test data: 0.001294665976178146
k = 7
Accuracy on test data: 0.0018125323666494044
k = 8
Accuracy on test data: 0.0018125323666494044
k = 9
Accuracy on test data: 0.0015535991714137752
k = 10
Accuracy on test data: 0.0010357327809425167
k = 11
Accuracy on test data: 0.001294665976178146
k = 12
Accuracy on test data: 0.001294665976178146
k = 13
Accuracy on test data: 0.001294665976178146
k = 14
Accuracy on test data: 0.0015535991714137752
k = 15
Accuracy on test data: 0.0018125323666494044
k = 16
Accuracy on test data: 0.0020714655618850335
k = 17
Accuracy on test data: 0.0020714655618850335
k = 18
Accuracy on test data: 0.002330398757120663
k = 19
Accuracy on test data: 0.0028482651475919213
k = 20
Accuracy on test data:

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import seaborn as sns

#print("Accuracy score: {}".format(accuracy_score(y_test, y_pred)))
#print("Mean squared error: {}".format(mean_squared_error(y_test, y_pred)))
a = confusion_matrix(y_test, y_pred)
#plt.imshow(a, cmap='hot')
#plt.show()

ax = sns.heatmap(a)
ax

In [ ]:
ax1 = sns.heatmap(op_data)
ax1

## BELOW I AM TRYING TO TO RUN KNN ON Y=oga AND X = DATASET

In [ ]:
# Splitting data into train and test datasets, with 7:3 ratio
y1 = op_data['oga'] #oga default
x1 = op_data
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.3, random_state=0)
print((x_train1.shape))
print(x_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
x_train = x_train1.as_matrix()
y_train = y_train1.as_matrix()
print((x_train.shape))
print((y_train.shape))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(x1,y1)
y_pred = knn.predict(x_test1)
print('k = {}'.format(k))
print('Accuracy on test data: {}'.format(knn.score(x_test1, y_test1)))

In [ ]:
#DETERMINING K NEAREST NEIGHBOURS!
#RUNNING KNeighborsClassifier FOR K VALUES 1 TO 30 TO SEE WHICH K VALUE RESULTS MAX ACCURACY
for k in range (1,30):
    knn = KNeighborsClassifier(n_neighbors=k) 
    knn.fit(x_train1,y_train1)
    y_pred = knn.predict(x_test1)
    print('k = {}'.format(k))
    print('Accuracy on test data: {}'.format(knn.score(x_test1, y_test1)))
    print('======================================================')

### BELOW CODE IS NOT FOR SUBMISSION
#### IT IS USED TO PARSE XML DATA FILES AND CONVERT THEM TO CSV BY ADDING A FAKE root NODE


In [ ]:
data = defaultdict(list)
parsedXML = etree.parse('operational_data/test_xmldata.2017-07-22.xml')
#parsedXML = etree.parse('operational_data/temp.xml')
#dfcols = ['devicename','timestamp', 'deviceid', 'incr', 'tokenid', 'seqnb']

root = parsedXML.getroot()
#df = pd.DataFrame()

def getvalueofnode( node ):
    return node.text if node is not None else None
time=0
condition=0
for node in parsedXML.getroot():
#     devicename = node.find('devicename')
#     df['devicename']= getvalueofnode(devicename)
#     timestamp = node.find('timestamp')
#     df['timestamp']= getvalueofnode(timestamp)
#     deviceid = node.find('deviceid')
#     df['deviceid']= getvalueofnode(deviceid)
#     incr = node.find('incr')
#     df['incr']= getvalueofnode(incr)
#     tokenid = node.find('tokenid')
#     df['tokenid']= getvalueofnode(tokenid)
#     seqnb = node.find('seqnb')
#     df['seqnb']= getvalueofnode(seqnb)
#ONLY THIS FOR LOOP IS ENOUGH AND THIS IS POSSIBLY CREATING A 1 ROW AND INFINITE COLUMN DATAFRAME NEED TO ADD TRY AND EXCEPT BLOCK
    if(node.tag=='objectdata'):
        for element in node.iter(): #root default
            temp = []
            try:
                if(element.tag == 'size'):
                    #print(element.attrib['ole'])
                    data['ole'].append(element.attrib['ole'])
                    data['owi'].append(element.attrib['owi'])
                    data['ohe'].append(element.attrib['ohe'])
                elif(element.tag=='otl'):
                    for elem in element:
                        data[element.tag].append(elem.text)
                elif(element.tag=='oga'):
                    for elem in element:
                        data[element.tag].append(elem.text)
                        #print(elem.text)
                elif(element.tag=='speed'):
                    for elem in element:
                        data[element.tag].append(elem.text)
                elif(element.tag=='timestamp'):
                    if(time==0):
                        data['exit_timestamp'].append(element.text)
                        time=1
                    else:
                        data['entry_timestamp'].append(element.text)
                        time=0
                else:
                    data[element.tag].append(element.text)
            except:
                if(element.tag == 'size'):
                    #print(element.attrib['ole'])
                    data['ole'] = (element.attrib['ole'])
                    data['owi'] = (element.attrib['owi'])
                    data['ohe'] = (element.attrib['ohe'])
                elif(element.tag=='otl'):
                    for elem in element:
                        data[element.tag] = (elem.text)
                elif(element.tag=='speed'):
                    for elem in element:
                        data[element.tag] = (elem.text)
                elif(element.tag=='oga'):
                    for elem in element:
                        data[element.tag] = (elem.text)
                elif(element.tag=='timestamp'):
                    if(time==0):
                        data['exit_timestamp'] = (element.text)
                        time=1
                    else:
                        data['entry_timestamp'] = (element.text)
                        time=0
                else:
                    data[element.tag] = element.text 




In [ ]:
#print((data))
result = pd.DataFrame.from_dict(data, orient='index')
result = result.transpose()
result= result[['devicename','entry_timestamp','exit_timestamp','ole','owi','ohe','speed','incr','otl','oga']]
result.to_csv('temp.csv',index=False)